# Tutorial 02-02 - Working with ArcGIS Pro Maps

Let's go back to our work with highway data for GeoNinjas PythonAnalytics.  

#### 1. Open the ArcGIS Pro Project

Although ArcGIS Pro projects can be comprised of a number of different files in a folder, when we reference them via arcpy we often will start with the **.aprx** file.  This is a good entry point that will let us access the data, maps, and layouts in the project.  We'll start this exercise by creating an **ArcGISProject** object using the .aprx file.

In [ ]:
import arcpy

# relative path from this script to the project
project_path = r".\Chapter 02 Files\Chapter 02 - Working with Maps.aprx"

# creating a project object
project = arcpy.mp.ArcGISProject(project_path)

#### 2.  Access the layout and elements

In this case, we happen to know that this project contains one layout that we want to modify.  Because of that, we can just accept the first layout returned from the **listLayouts** method.  If we had multiple layouts in the same project, we could use this method to search and/or iterate through the results.

In [ ]:
layout = project.listLayouts()[0]
layout.name

To access individual elements in the layout, we can call the **listElements** method.  This will return any elements in the layout including text boxes, map frames, North arrows, charts, and more.  

In [ ]:
for element in layout.listElements():
    print(element.name)

Since this is a fairly simple layout, we can just access the elements we want to modify by name.  In our case, we'll be doing some filtering in the *Map Frame* element and we'll change the text in the *Text* element.  We can access both of those by using list comprehensions.  Alternatively, you could just write a for-loop and iterate through all the elements until you find the one you want.

In [ ]:
title = [e for e in layout.listElements() if e.name == 'Text'][0]
map_frame = [e for e in layout.listElements() if e.name == 'Map Frame'][0]

TODO?  do we wnat to explain list comprehensions here?

#### 3. Fix Broken Layers

If you've already opened the ArcGIS Pro project in this folder, you may have noticed that there are two "broken" layers.  The "Counties" and "Highways" layers in the Map are referencing data sources with bad paths.  This happens less often than it used to in the days of ArcMap, but it's still something we can look for using arcpy.

In [ ]:
for layer in map_frame.map.listLayers():
    if layer.name in ['Highways_Intersect','Counties']:
        print(layer.name, layer.isBroken)

Not only can we identify this issue using arcpy, we can also fix it.  In this case, the data we need is in the project database.  We can iterate through each of the layers, get the connection properties, and modify those properties to read from the project database

In [ ]:
# iterate through each layer
for layer in map_frame.map.listLayers():
    
    # check only the layers we think are broken
    if layer.name in ['Highways_Intersect','Counties']:
        
        # get a copy the connection properties of the broken layer
        layer_conn_props = layer.connectionProperties
        
        # replace the database path in the copy of the connection properties
        layer_conn_props['connection_info']['database'] = project.databases[0]['databasePath']
        
        # use the copy we fixed to update the layer
        layer.updateConnectionProperties(
            layer.connectionProperties, layer_conn_props
        )
        print(layer.name, layer.isBroken)

Now that we've fixed our project, we can save and move on.

In [ ]:
project.save()

#### 4.  Work with the layers in the map view

Now that we've fixed the broken layers in our map, we can work on filtering the datasets.  In ArcGIS Pro, there's a concept of *definition queries* on layers.  These are SQL where clauses that can be used to filter the records or features that are shown in a layer.

We can iterate through both our highways and counties layers and set the definition query to use the *NAMELSAD* field to our county of interest.

In [ ]:
# pick a county to test with
county = 'Alameda County'

# set the definition query for the counties and highways layers
counties_layer.definitionQuery = f"NAMELSAD = '{county}'"
highways_layer.definitionQuery = f"NAMELSAD = '{county}'"

We'll also want to set the extent of our map to the county.  This way we're zoomed to just the data that we're concerned about and not viewing the entire state.  We can get that extent by calling the **getLayerExtent** method of the MapFrame object (map_frame).  We can then set the MapFrame's extent to that layer extent.

In [ ]:
# get the county extent
county_extent = map_frame.getLayerExtent(counties_layer)

# set the MapFrame's extent
map_frame.camera.setExtent(county_extent)

#### 5.  Modify layout and print to PDF

Now that we've filtered our data for a single county and set the extent for our map, we can export the map to a print-friendly PDF.  Before we do this, we can modify our map's title by working with the text element we identified in Step 2.

In [ ]:
# set the title text
title.text = county

Now to export our map to PDF, we can call the **exportToPDF** method on our layout.

In [ ]:
# export the PDF.  Name the file using the county name.
layout.exportToPDF(f'./{county}.pdf')

#### 6.  Repeat the process for multiple counties

Since we've worked out the logic for modifying the map and printing for one county, we can now reuse that code to export multiple counties.  Let's take the code we developed in our previous steps and put it in a for loop.

In [ ]:
# list of counties to export
counties = [
    "Alameda County",
    "Marin County",
    "Contra Costa County",
    "San Francisco County",
    "Santa Clara County"
]

In [ ]:
# iterate through the counties
for county in counties:
    # set the definition query for the counties and highways layers
    counties_layer.definitionQuery = f"NAMELSAD = '{county}'"
    highways_layer.definitionQuery = f"NAMELSAD = '{county}'"
    
    # get the county extent
    county_extent = map_frame.getLayerExtent(counties_layer)

    # set the MapFrame's extent
    map_frame.camera.setExtent(county_extent)
    
    # set the title text
    title.text = county
    
    # export the PDF.  Name the file using the county name.
    layout.exportToPDF(f'./{county}.pdf')